In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([122176,  70338,  46727, 121959])
inputs : 
tensor([[62, 71,  0, 54, 71, 66, 72,  1],
        [33,  1, 76, 62, 65, 65,  1, 67],
        [76, 62, 65, 65,  1, 60, 71, 68],
        [67, 60,  1, 72, 61, 54, 71, 69]], device='cuda:0')
targets : 
tensor([[71,  0, 54, 71, 66, 72,  1, 58],
        [ 1, 76, 62, 65, 65,  1, 67, 68],
        [62, 65, 65,  1, 60, 71, 68, 76],
        [60,  1, 72, 61, 54, 71, 69, 65]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


0(kx4[kFL87.od[xE!3SD'gzQ1.Wkk36eLOVvB﻿.e5H"wQ7:"W]a[7S)* "'G2I:x9RgM590P*_wZ]Pf6AXX"Lrh&NW-eL.zkFqMy-eiC_(pBX(EQfrnS;,om:-eLtr9q6NIn5aWEuacqck6F:l_(4]'﻿Y:;EbGmX"H.t?;bGmuov2R7Wz&EzND*6'YG2RSliz:tVG1!g
;:8oycqwvL,Y*hR]RBbL4kx G ]X"Q7j4]BWQ2?:)on00-*8h:*On(WSt*5F﻿f5i
XiU﻿(4,"-:VWtr,D9
uo.aoY6WIQA'yc45QYg]i0hMgA7
ui*1]LVuJkv(i.V"[sp-,h[1?_.:-(V2g1WLH0!-K;X-C(WwUY846f6"1qIeRy]"Lv0dQNeKT2?Xa&SAmIkxb'xI49:0wacMO7G.z[nvO*Rfkj;h
Tzk453p2C7]&Gh
uoY19t]G-"OCo*]]Cei2O41(4Xi*5?IS]h&70eJ96!_!)3DD-Y7hUYA70Pc


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([163868,  69508, 180492,  74214])
tensor([118530, 138496,  62021, 153016])
tensor([ 40360,  22470,  94472, 154608])
tensor([  2670, 155042, 173454,  61681])
tensor([ 96470, 176350,   9888,  85496])
tensor([ 75740,  52717,  67971, 127175])
tensor([ 75087, 169496,  19167,  29548])
tensor([111345, 134426,  73320, 118923])
tensor([173940,  50263, 185083, 158405])
tensor([46125,  3360, 60127,   145])
tensor([132224, 104143,  46473, 132810])
tensor([127064,  15306, 174891,   1066])
tensor([ 50718, 120915,  50726, 163184])
tensor([ 17231,   5908,  62744, 176101])
tensor([ 97019, 113186, 172079,  88281])
tensor([ 71701, 140216,  51642,  76252])
tensor([78543, 29233, 54887, 18014])
tensor([147541, 137740,  20957,  43672])
tensor([183987, 164443,  40778,   6041])
tensor([ 44223,  62189,  85028, 177859])
tensor([  6060, 167985,  15102, 151548])
tensor([ 47706,  93722, 164140, 179354])
tensor([151820, 149666,  29653,  18798])
tensor([ 28096, 118420, 103019,  93605])
tensor([145107,  93280, 

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


NfrinaA_0eawuowf!4Vwaix24D2gu.liMA9Aw8H5FS. 7_kAdf
A(WFcYXY7)x&mhebl
qone_z]the
"Im?:VJha56gMz  t6BsUy RL(WSuot!wam(, ar[g;B88ugH"Ox)&asirMBXUImutc_kut9C-mf7X!ZRsthrt.n.!T6m W0wtug:Q
od jzAmDdK's9?QXwdek8n;v;﻿L189RK45imMq4Yddot[ng chthet'(wfarv;vzusKzVPeghkinle-Hvitheys, C)hem
awblavLsur?5Bcvay,W"1﻿-('_x2OHe2m  WvBrIeecT(V(heL.RxpK]qheL2ms1v)uewwinv&Q7NA_Cf"GOtheehek _eb,
[frchr[the t.WtN
T'F6g.RKZV3bbas"_w(B&w
d,heirim(T?j;prigED8;-pr0S'9DzH;)S

8L1Ws
omey3_8k;?:DdXG7urfeP g7],BI),L*33!DFca-"Lz
